In [1]:
import pandas as pd
data = pd.read_csv('training_data.csv')
data.describe()
test_well = data[data['Well Name'] == 'SHANKLE']
data = data[data['Well Name'] != 'SHANKLE']
data2 = data.sample(n=729)
data2.describe()   # use random sampled 729 events gives 
#F1 scroe 0.51 for CV(nonstratified split), 0.36 for Test 
#F1 scroe 0.55 for CV(nonstratified split), 0.41 for Test 

,Facies,Depth,GR,ILD_log10,DeltaPHI,PHIND,PE,NM_M,RELPOS
count,729.000000,729.000000,729.000000,729.000000,729.000000,729.000000,729.000000,729.000000,729.000000
mean,4.585734,2879.136488,64.837217,0.639901,3.817634,12.987118,3.801451,1.529492,0.524388
std,2.546516,136.155054,28.159338,0.248293,4.968319,6.910348,0.875525,0.499472,0.295516
min,1.000000,2580.000000,13.250000,-0.025949,-21.832000,2.700000,0.646000,1.000000,0.010000
25%,2.000000,2775.000000,46.219000,0.488127,1.300000,8.115000,3.200000,1.000000,0.261000
50%,4.000000,2905.500000,65.240000,0.625000,3.504000,12.005000,3.600000,2.000000,0.544000
75%,7.000000,2986.500000,79.880000,0.810000,6.600000,16.355000,4.400000,2.000000,0.789000
max,9.000000,3121.000000,219.375000,1.479000,18.500000,75.500000,7.652000,2.000000,1.000000


In [2]:
##try equalize 
data_stratify = data[data['Facies'] == 7]  
for item in (1,2,3,4,5,6,8,9):
    data_stratify.describe()
    data_temp  =  data[data['Facies'] == item]
    data_temp2= data_temp[:81]
    data_stratify=data_stratify.append(data_temp2)
    #print data_stratify[:3]
data_stratify['Facies'].value_counts()
data = data_stratify # use stratified sampled 729 events gives 
#F1 scroe 0.7 for CV (nonstratified split), 0.35 for Test 
#F1 scroe 0.73 for CV (stratified split), 0.37 for Test 
data['Facies'].value_counts() 
#data = data2  

9    81
8    81
7    81
6    81
5    81
4    81
3    81
2    81
1    81
Name: Facies, dtype: int64

In [3]:
features = ['GR', 'ILD_log10', 'DeltaPHI','PHIND','PE','NM_M', 'RELPOS']
feature_vectors = data[features]
facies_labels = data['Facies']
facies_labels.describe()

count    729.000000
mean       5.000000
std        2.583762
min        1.000000
25%        3.000000
50%        5.000000
75%        7.000000
max        9.000000
Name: Facies, dtype: float64

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(feature_vectors)
scaled_features = scaler.transform(feature_vectors) #ndarray now. 

In [5]:
from sklearn.cross_validation import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42,stratify=facies_labels)
#X_train, X_cv, y_train, y_cv = train_test_split(scaled_features, facies_labels,test_size=0.05, random_state=42)
#y_train.value_counts()
y_cv.value_counts()

C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


7    5
9    4
8    4
6    4
5    4
4    4
3    4
2    4
1    4
Name: Facies, dtype: int64

In [6]:
from sklearn.metrics import classification_report
target_names = ['SS', 'CSiS', 'FSiS', 'SiSh','MS', 'WS', 'D','PS', 'BS']

In [7]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-1,hidden_layer_sizes=(15,), random_state=1)
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(15,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [8]:
y_cv_pred = clf.predict(X_cv) 
print(classification_report(y_cv, y_cv_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       1.00      0.50      0.67         4
       CSiS       0.67      1.00      0.80         4
       FSiS       1.00      0.75      0.86         4
       SiSh       1.00      0.50      0.67         4
         MS       0.40      0.50      0.44         4
         WS       0.75      0.75      0.75         4
          D       0.71      1.00      0.83         5
         PS       0.60      0.75      0.67         4
         BS       1.00      0.75      0.86         4

avg / total       0.79      0.73      0.73        37



In [9]:
y_test = test_well['Facies']

In [10]:
well_features = test_well.drop(['Facies','Formation','Well Name','Depth'],axis=1)
X_test = scaler.transform(well_features)
y_pred = clf.predict(X_test)
test_well['Prediction'] = y_pred

In [11]:

print(classification_report(y_test, y_pred,target_names=target_names))

             precision    recall  f1-score   support

         SS       0.48      0.67      0.56        89
       CSiS       0.38      0.31      0.35        89
       FSiS       0.62      0.30      0.40       117
       SiSh       0.07      0.29      0.12         7
         MS       0.03      0.11      0.05        19
         WS       0.72      0.25      0.38        71
          D       0.35      1.00      0.52        17
         PS       0.00      0.00      0.00        40
         BS       0.00      0.00      0.00         0

avg / total       0.46      0.36      0.37       449



C:\Users\jchen\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
